In [2]:
import numpy as np
import pandas as pd
import json
import os
import sklearn as sk

# 读取

In [3]:
BNAcap_393 = pd.read_csv("BNACAP 明确病原体-393例-xjy20190721.csv", header=0,index_col="病例入组编号")
BNAcap_954 = pd.read_csv("BNACAP-954例（排除结核）纳入病原谱文章分析 分组分析用 加评分.csv", header=0,index_col="病例入组编号")

# 入院信息

In [ ]:
inpatients = json.load(open("inpatients.json", "r"))
print(inpatients.keys())
BNAcap_954[inpatients["present_info"]]

# SOFA 分组

In [30]:
SOFA_col = ["PaO2mmHg", "血气分析FiO2","Plt109L","TBLIμmolL","呼吸支持方式","氧疗","无创通气","有创通气",
"收缩压","舒张压","应用1","应用2","血管活性药物","CrμmolL","神经系统症状"]
SOFA_df = BNAcap_954[SOFA_col]

In [ ]:
# 呼吸评分
rep_l = {}
for i in SOFA_df.index:
    
    if  SOFA_df.loc[i, "PaO2mmHg"] != " ":
        if SOFA_df.loc[i, "血气分析FiO2"] != " ":
            ox = float(SOFA_df.loc[i, "PaO2mmHg"])/(float(SOFA_df.loc[i, "血气分析FiO2"])/100)
        else:
            ox = float(SOFA_df.loc[i, "PaO2mmHg"])/(21/100)

        if ox>=400:
            cla = 0
        elif ox>=300:
            cla = 1
        elif ox>=200:
            cla = 2
        elif ox>=100:
            cla = 3
        else:
            cla = 4
        
        if SOFA_df.loc[i, "呼吸支持方式"] != " ":
            if ox >=100:
                cla = 3
            else:
                cla = 4
         
    else:
        if SOFA_df.loc[i, "呼吸支持方式"] != " ":
                cla = 3
        else:
            cla = 0
    
    rep_l[i] = cla
# 循环
bp_l = {}
for i in SOFA_df.index:
    if SOFA_df.loc[i, "收缩压"] == " " or SOFA_df.loc[i, "舒张压"] == " ":
        cla = 0
    else:
        map = int(SOFA_df.loc[i, "舒张压"]) + (int(SOFA_df.loc[i, "收缩压"]) - int(SOFA_df.loc[i, "舒张压"]))/3
        if SOFA_df.loc[i, "血管活性药物"] == 1:
            cla = 2
        else:
            if map>=70:
                cla = 0
            else:
                cla = 1
    bp_l[i] = cla
# 泌尿
rena_l = {}
for i in SOFA_df.index:
    if SOFA_df.loc[i, "CrμmolL"] == " ":
        cla = 0
    else:
        if float(SOFA_df.loc[i, "CrμmolL"]) < 110:
            cla = 0
        elif float(SOFA_df.loc[i, "CrμmolL"]) <= 170:
            cla = 1
        elif float(SOFA_df.loc[i, "CrμmolL"]) <= 299:
            cla = 2
        elif float(SOFA_df.loc[i, "CrμmolL"]) <= 440:
            cla = 3
        else:
            cla = 4
    rena_l[i] = cla
# 肝脏
liver_l = {}
for i in SOFA_df.index:
    if SOFA_df.loc[i, "TBLIμmolL"] == " ":
        cla = 0
    else:
        if float(SOFA_df.loc[i, "TBLIμmolL"]) < 20:
            cla = 0
        elif float(SOFA_df.loc[i, "TBLIμmolL"]) <= 32:
            cla = 1
        elif float(SOFA_df.loc[i, "TBLIμmolL"]) <= 101:
            cla = 2
        elif float(SOFA_df.loc[i, "TBLIμmolL"]) <= 204:
            cla = 3
        else:
            cla = 4
    liver_l[i] = cla
# 凝血
plt_l = {}
for i in SOFA_df.index:
    if SOFA_df.loc[i, "Plt109L"] == " ":
        cla = 0
    else:
        if float(SOFA_df.loc[i, "Plt109L"]) >= 150:
            cla = 0
        elif float(SOFA_df.loc[i, "Plt109L"]) > 100:
            cla = 1
        elif float(SOFA_df.loc[i, "Plt109L"]) > 50:
            cla = 2
        elif float(SOFA_df.loc[i, "Plt109L"]) > 20:
            cla = 3
        else:
            cla = 4
    plt_l[i] = cla
# 凝血
mental_l = {}
for i in SOFA_df.index:
    if SOFA_df.loc[i, "神经系统症状"] == " ":
        cla = 0
    else:
        if SOFA_df.loc[i, "神经系统症状"] == 0:
            cla = 0
        else:
            cla = 1
    mental_l[i] = cla 

In [ ]:
sofa_socre = {}
for i in SOFA_df.index:
    sofa_socre[i] = rep_l[i] + bp_l[i] + rena_l[i] + liver_l[i] + plt_l[i] + mental_l[i]
    print(sofa_socre[i])

In [31]:
virus_label = BNAcap_393.loc[:,["支原体",
"支原体培养",
"甲型流感病毒",
"腺病毒",
"偏肺病毒",
"鼻病毒",
"呼吸道合胞病毒A型",
"呼吸道合胞病毒B型",
"冠状病毒OC43HKU1",
"冠状病毒229ENL63",
"副流感病毒1型",
"副流感病毒2型",
"副流感病毒3型",
"副流感病毒4型",
"乙型流感病毒",
"博卡病毒",
"肠道病毒"]]